In [ ]:
import pymongo
import spacy
import nltk
import string
import networkx as nx
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Télécharger les stopwords français
nltk.download("punkt")
nltk.download("stopwords")
stop_words = set(stopwords.words("french"))
# Charger le modèle SpaCy pour le français
nlp = spacy.load("fr_core_news_sm")
# Connexion à MongoDB
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["Coursdb"]  # Nom de ta base de données

# Liste des collections à traiter (ex: modules de cours)
modules = db.list_collection_names()
modules
def nettoyer_texte(texte):
    """ Nettoie le texte : suppression de la ponctuation, mise en minuscules et suppression des stopwords. """
    texte = texte.lower().translate(str.maketrans("", "", string.punctuation))
    tokens = word_tokenize(texte, language="french")
    tokens = [mot for mot in tokens if mot not in stop_words and len(mot) > 2]  # Suppression des mots vides et courts
    return " ".join(tokens)


def extraire_mots_cles_tfidf(documents):
    """ Applique TF-IDF pour extraire les mots-clés les plus pertinents dans un corpus de documents. """
    corpus = [doc["content"] for doc in documents]
    
    # Vérification des documents non vides
    corpus_non_vide = [texte for texte in corpus if len(texte.strip()) > 0]  # Filtrer les documents vides

    if not corpus_non_vide:
        return []  # Retourner une liste vide si tous les documents sont vides
    
    vectorizer = TfidfVectorizer(max_features=10)  # Extraire les 10 mots les plus importants
    tfidf_matrix = vectorizer.fit_transform(corpus_non_vide)
    mots_cles = vectorizer.get_feature_names_out()
    return mots_cles


def extraire_concepts(texte, taille_max=1_000_000):
    """ Divise un texte en morceaux et applique SpaCy sur chaque partie. """
    nlp.max_length = 10_000_000  # Augmente la limite si nécessaire
    concepts = set()
    
    # Découper le texte en morceaux
    for i in range(0, len(texte), taille_max):
        morceau = texte[i:i + taille_max]
        doc = nlp(morceau)
        for ent in doc.ents:
            concepts.add(ent.text)

    return list(concepts)


def creer_graphe_connaissance(concepts):
    """ Crée un graphe reliant les concepts extraits. """
    graphe = nx.Graph()
    for i, concept1 in enumerate(concepts):
        for j, concept2 in enumerate(concepts):
            if i != j:  # On évite de relier un concept à lui-même
                graphe.add_edge(concept1, concept2)
    return graphe


# Traitement des documents dans chaque module
for module in modules:
    collection = db[module]  # Accéder à la collection du module
    documents = list(collection.find({}))  # Récupérer tous les documents

    for doc in documents:
        texte = nettoyer_texte(doc["content"])

        # Extraction des concepts
        mots_cles_tfidf = extraire_mots_cles_tfidf([doc])
        entites_spacy = extraire_concepts(texte)

        # Fusion des mots-clés et entités pour obtenir une meilleure liste de concepts
        concepts_extraits = list(set(mots_cles_tfidf).union(set(entites_spacy)))

        # Limiter la taille des concepts et relations pour éviter le dépassement de taille
        concepts_extraits = concepts_extraits[:100]  # Limiter à 100 concepts maximum
        graphe = creer_graphe_connaissance(concepts_extraits)

        # Limiter les relations à 100
        relations = [{"concept1": edge[0], "relation": "lié à", "concept2": edge[1]} for edge in graphe.edges][:100]

        # Mise à jour du document dans MongoDB
        collection.update_one(
            {"_id": doc["_id"]},
            {"$set": {"concepts": concepts_extraits, "relations": relations}}
        )

    print(f"Traitement terminé pour le module : {module}")

print("Analyse terminée, données mises à jour dans MongoDB.")
